In [1]:
import numpy as np
import pandas as pd

# train_df = pd.read_json('../train.json', encoding='utf8', lines=True)
train_df = pd.read_csv('../train_xlnet.tsv', encoding='utf8', sep='\t')

train_df['text'] = train_df['text'].apply(lambda x: x.replace('<sep>', '_'))
train_trans_back_df = pd.read_csv('../translation/zh-CN/trainA_trans.tsv', encoding='utf8', sep='\t')
train_trans_back_df['text'] = train_trans_back_df['text'].apply(lambda x: x.replace('<sep>', '_'))

train_df = pd.concat((train_df, train_trans_back_df), axis=0).sort_values('label_id')
# train_df.to_csv('train_backtrans_prompt.tsv', encoding='utf8', sep='\t', index=None)
train_df

,label_id,text
0,0,一种信号的发送方法及基站、用户设备_一种信号的发送方法及基站、用户设备。在一个子帧中为多个用...
1,0,一种5G通信电缆故障监测系统_本发明公开了一种5G通信电缆故障监测系统，包括信号采样模块和补...
2,0,测试方法及电子装置_本发明提供一种测试方法及电子装置。该方法包括以下步骤：根据选择的测试任务...
3,0,天线方位调整装置_一种天线方位角调节装置，包括两个相对的8字形支架（101）、动力输入电机（...
4,0,光纤老化预测方法及装置_本申请提供一种光纤老化预测方法及其装置。该方法包括：获取待测光纤模块...
...,...,...
955,35,一种广告投放控制方法及装置_本发明公开了一种广告投放控制方法及装置，以解决现有技术中基于地域...
954,35,一种适用于安防的广告机_本实用新型公开了一种适用于安防的广告机，包括支撑架，支撑架的上端设置...
953,35,一种信息处理方法及装置_本发明公开了一种信息处理方法，所述方法包括：第一进程获取来自多个查询...
960,35,本发明涉及一种广告投放控制方法及装置_本发明公开了一种广告投放控制方法及装置，用于解决现有技...


In [2]:
token_sep = '/'
label_names_df = pd.read_csv('../labels/named_labels.tsv', sep='\t', encoding='utf8')
label_names_df['name'] = label_names_df['name_0'] + token_sep + label_names_df['name_1'] + token_sep + label_names_df['name_2']

label_names = np.asarray(label_names_df['name']).tolist()

label_names_df

,label_id,name_0,name_1,name_2,name
0,0,通信,信号,天线,通信/信号/天线
1,1,农业,基因,养殖,农业/基因/养殖
2,2,服装,基建,机械,服装/基建/机械
3,3,存储,数据,信息,存储/数据/信息
4,4,精算,终端,交通,精算/终端/交通
5,5,材料,化学,生物,材料/化学/生物
6,6,智能,科技,算法,智能/科技/算法
7,7,监测,杀菌,环保,监测/杀菌/环保
8,8,医疗,疾病,细胞,医疗/疾病/细胞
9,9,图像,视频,画面,图像/视频/画面


In [3]:
# train_df['input'] = "这是哪种类别的专利？_" + train_df['title'].str.replace('<i>|</i>|"', '', regex=True) + "。" + train_df['abstract'].str.replace('<i>|</i>|"', '', regex=True) + '_选项：' + ",".join([l for l in label_names]) + "_答案："
train_df['input'] = "这是哪种类别的专利？_" + train_df['text'].str.replace('<i>|</i>|"', '', regex=True) + '_选项：' + ",".join([l for l in label_names]) + "_答案："
train_df['target'] = train_df['label_id']
train_df[['input', 'target']].to_csv("train_prompt.tsv", encoding='utf8', sep='\t', index=None)
train_df[['input', 'target']]

,input,target
0,这是哪种类别的专利？_一种信号的发送方法及基站、用户设备_一种信号的发送方法及基站、用户设备...,0
1,这是哪种类别的专利？_一种5G通信电缆故障监测系统_本发明公开了一种5G通信电缆故障监测系统...,0
2,这是哪种类别的专利？_测试方法及电子装置_本发明提供一种测试方法及电子装置。该方法包括以下步...,0
3,这是哪种类别的专利？_天线方位调整装置_一种天线方位角调节装置，包括两个相对的8字形支架（1...,0
4,这是哪种类别的专利？_光纤老化预测方法及装置_本申请提供一种光纤老化预测方法及其装置。该方法...,0
...,...,...
955,这是哪种类别的专利？_一种广告投放控制方法及装置_本发明公开了一种广告投放控制方法及装置，以...,35
954,这是哪种类别的专利？_一种适用于安防的广告机_本实用新型公开了一种适用于安防的广告机，包括支...,35
953,这是哪种类别的专利？_一种信息处理方法及装置_本发明公开了一种信息处理方法，所述方法包括：第...,35
960,这是哪种类别的专利？_本发明涉及一种广告投放控制方法及装置_本发明公开了一种广告投放控制方法...,35


In [4]:
testA_df = pd.read_json('../testA.json', encoding='utf8', lines=True)
testA_df['input'] = "这是哪种类别的专利？_" + testA_df['title'].str.replace('<i>|</i>|"', '', regex=True) + "。_" + testA_df['abstract'].str.replace('<i>|</i>|"', '', regex=True) + '_选项：' + ",".join([l for l in label_names]) + "_答案："
testA_df['target'] = pd.read_csv('../../results/62/4/submit (6452_bactrans_oversampling24_1_5epochs_rdrop result_0 manually 6452).csv', encoding='utf8')['label']
testA_df[['input']].to_csv("testA_prompt.tsv", encoding='utf8', sep='\t', index=None)
testA_df[['input', 'target']].to_csv("testA_prompt_target.tsv", encoding='utf8', sep='\t', index=None)
train_df = pd.DataFrame(train_df[['input', 'target']])
testA_df = pd.DataFrame(testA_df[['input', 'target']])

output = pd.concat((train_df, testA_df), axis=0).sort_values('target')
output['target'] = output['target'].apply(lambda x: label_names[x])

suffix = '_选项：' + ",".join([l for l in label_names]) + "_答案："
output['input'] = output['input'].apply(lambda x: (x[:1024 - len(suffix)] + suffix if len(x) > 1024 else x))
output.to_csv('train6452_prompt.tsv', sep='\t', encoding='utf8', index=None)

output.reset_index()

,input,target
0,这是哪种类别的专利？_一种信号的发送方法及基站、用户设备_一种信号的发送方法及基站、用户设备...,通信/信号/天线
13547,这是哪种类别的专利？_一种用于配置无线路由设备的方法与设备。_本申请的目的是提供一种用于配置...,通信/信号/天线
18183,这是哪种类别的专利？_用于上行链路多用户传输的ACK/NACK信号处理方法及装置。_本文件涉...,通信/信号/天线
13557,这是哪种类别的专利？_一种便于安装和使用的路由器。_本实用新型公开了一种便于安装和使用的路由...,通信/信号/天线
18180,这是哪种类别的专利？_一种哈希冲突的处理方法、装置及交换设备。_本发明提供了一种MAC地址哈...,通信/信号/天线
...,...,...
14814,这是哪种类别的专利？_广告推广处理方法及装置。_本发明实施例提供的一种广告推广处理方法及装置...,数据/广告/信息
11853,这是哪种类别的专利？_媒体文件的处理方法、装置和广告分析方法。_本发明公开了一种媒体文件的处...,数据/广告/信息
18011,这是哪种类别的专利？_一种在影院推送广告的方法。_本发明是关于一种在影院推送广告的方法。该方...,数据/广告/信息
958,这是哪种类别的专利？_基于大数据的广告推送系统_本发明公开了一种基于大数据的广告推送系统，包...,数据/广告/信息


In [5]:
input_list = np.asarray(output['input']).tolist()
l = [len(i) for i in input_list]
l.sort(reverse=True)
l

[1024,
 1024,
 991,
 946,
 910,
 907,
 896,
 889,
 875,
 861,
 861,
 855,
 854,
 844,
 844,
 841,
 833,
 829,
 820,
 818,
 818,
 815,
 808,
 807,
 807,
 804,
 802,
 802,
 796,
 793,
 792,
 788,
 788,
 786,
 785,
 783,
 781,
 780,
 779,
 779,
 776,
 776,
 775,
 774,
 774,
 773,
 771,
 770,
 769,
 768,
 768,
 767,
 767,
 767,
 766,
 766,
 766,
 766,
 765,
 765,
 764,
 763,
 763,
 763,
 762,
 758,
 758,
 757,
 757,
 757,
 756,
 756,
 755,
 755,
 754,
 754,
 754,
 753,
 753,
 751,
 751,
 751,
 751,
 750,
 750,
 748,
 748,
 747,
 747,
 747,
 747,
 747,
 746,
 746,
 746,
 746,
 746,
 746,
 744,
 744,
 743,
 743,
 743,
 742,
 742,
 742,
 741,
 741,
 741,
 741,
 740,
 740,
 740,
 740,
 740,
 740,
 739,
 737,
 737,
 737,
 737,
 737,
 737,
 736,
 736,
 736,
 736,
 735,
 735,
 735,
 734,
 734,
 734,
 734,
 733,
 733,
 732,
 732,
 732,
 732,
 731,
 731,
 731,
 731,
 730,
 730,
 730,
 730,
 730,
 730,
 729,
 729,
 729,
 729,
 728,
 728,
 728,
 728,
 728,
 727,
 727,
 727,
 727,
 727,
 727,
 726,
 7

In [2]:
label_list = [
    "通信/信号/天线",
    "农业/基因/养殖",
    "服装/基建/机械",
    "存储/数据/信息",
    "精算/终端/交通",
    "材料/化学/生物",
    "智能/科技/算法",
    "监测/杀菌/环保",
    "医疗/疾病/细胞",
    "图像/视频/画面",
    "光电/电子/元件",
    "焊接/机械/制造",
    "传感/显示/三维",
    "冶炼/化合/制备",
    "石油/降解/回收",
    "电力/设备/电缆",
    "光伏/发电/配件",
    "电池/驱动/工装",
    "无机/制备/涂层",
    "风电/监测/控制",
    "药物/合成/生物",
    "复合/纳米/超导",
    "建筑/测定/试验",
    "钢材/腐蚀/生产",
    "车辆/轨道/列车",
    "聚合/纤维/材料",
    "导航/接收/航天",
    "矿产/水利/岩土",
    "烧结/锅炉/洗砂",
    "能源/充电/调度",
    "食品/水产/加工",
    "技术/装置/系统",
    "设备/方法/装置",
    "海洋/开采/机械",
    "航空/航天/控制",
    "数据/广告/信息"
]

In [21]:
import pandas as pd

testA_df = pd.read_json('../testA.json', encoding='utf8', lines=True)
testA_labels_df = pd.read_csv('../../results/prompt/4/predictions.csv', encoding='utf8', sep='\t', header=0, names=['label_names'])
# testA_labels_df[testA_labels_df['label_names'] == '汽车&轨道&列车'] = '车辆&轨道&列车'
# testA_labels_df[testA_labels_df['label_names'] == '建筑材料&化合&制备'] = '冶炼&化合&制备'
# testA_labels_df[testA_labels_df['label_names'] == '测绘&杀菌&环保'] = '监测&杀菌&环保'
# testA_labels_df[testA_labels_df['label_names'] == '测绘&终端&交通'] = '精算&终端&交通'
# testA_labels_df[testA_labels_df['label_names'] == '建筑材料&纤维&材料'] = '聚合&纤维&材料'
# testA_labels_df[testA_labels_df['label_names'] == '船舶&轨道&列车'] = '车辆&轨道&列车'
# testA_labels_df[testA_labels_df['label_names'] == '有机硅改性脂环族环氧丙烯酸酯8～25%;超支化预聚物4%～30%;丙烯酸酯活性稀释剂4～20%;脂环族环氧活性稀释剂50～80%;自由基光引发剂0.5～8%;大分子硫盐阳离子光引发剂Esacure11871～8%;光固化流平剂0.3～3%;阻聚剂OmnistabIC0.05%～1%;大分子活性抗氧剂ChinoxGM0.5%～5%'] = '无机&制备&涂层'
testA_labels_df[testA_labels_df['label_names'] == '电子/元件'] = '光电/电子/元件'
testA_labels_df[testA_labels_df['label_names'] == '信号/天线'] = '通信/信号/天线'
testA_labels_df[testA_labels_df['label_names'] == '风电/监测/杀菌/环保'] = '风电/监测/控制'
testA_labels_df[testA_labels_df['label_names'] == '铸坯加热'] = '钢材/腐蚀/生产'
testA_labels_df[testA_labels_df['label_names'] == '铸坯'] = '钢材/腐蚀/生产'
testA_labels_df[testA_labels_df['label_names'] == '化学/生物'] = '材料/化学/生物'
testA_labels_df[testA_labels_df['label_names'] == '科技/算法'] = '智能/科技/算法'
testA_labels_df[testA_labels_df['label_names'] == '铸钢'] = '钢材/腐蚀/生产'
testA_labels_df[testA_labels_df['label_names'] == '机械/制造'] = '焊接/机械/制造'
testA_labels_df[testA_labels_df['label_names'] == '画面'] = '图像/视频/画面'
testA_labels_df[testA_labels_df['label_names'] == '生物'] = '材料/化学/生物'
testA_labels_df[testA_labels_df['label_names'] == '钛合金'] = '冶炼/化合/制备'
testA_labels_df[testA_labels_df['label_names'] == '电缆'] = '电力/设备/电缆'
testA_labels_df[testA_labels_df['label_names'] == '交通'] = '精算/终端/交通'
testA_labels_df[testA_labels_df['label_names'] == '机械'] = '焊接/机械/制造'
testA_labels_df[testA_labels_df['label_names'] == '天线'] = '通信/信号/天线'
testA_labels_df[testA_labels_df['label_names'] == '发电/配件'] = '光伏/发电/配件'
testA_labels_df[testA_labels_df['label_names'] == '土壤/处理池'] = '矿产/水利/岩土'
testA_labels_df[testA_labels_df['label_names'] == '工装'] = '服装/基建/机械'
testA_labels_df[testA_labels_df['label_names'] == '马/腐蚀/生产'] = '钢材/腐蚀/生产'
testA_labels_df[testA_labels_df['label_names'] == '铝合金/轮毂'] = '冶炼/化合/制备'
testA_labels_df[testA_labels_df['label_names'] == '环保'] = '监测/杀菌/环保'
testA_labels_df[testA_labels_df['label_names'] == '疾病/细胞'] = '医疗/疾病/细胞'
testA_labels_df[testA_labels_df['label_names'] == '水利/岩土'] = '矿产/水利/岩土'
testA_labels_df[testA_labels_df['label_names'] == '列车'] = '车辆/轨道/列车'
testA_labels_df[testA_labels_df['label_names'] == '轨道/列车'] = '车辆/轨道/列车'
testA_labels_df[testA_labels_df['label_names'] == '马氏体'] = '钢材/腐蚀/生产'
testA_labels_df[testA_labels_df['label_names'] == '算法'] = '智能/科技/算法'
testA_labels_df[testA_labels_df['label_names'] == '冶炼/化学/生物'] = '冶炼/化合/制备'
testA_labels_df[testA_labels_df['label_names'] == '马氏体钢'] = '钢材/腐蚀/生产'
testA_labels_df[testA_labels_df['label_names'] == '马氏体/金属材料'] = '钢材/腐蚀/生产'
testA_labels_df[testA_labels_df['label_names'] == '钛铝合金材料'] = '冶炼/化合/制备'
testA_labels_df[testA_labels_df['label_names'] == '铝合金/腐蚀/生产'] = '钢材/腐蚀/生产'
testA_labels_df[testA_labels_df['label_names'] == '铝合金/化学/生物'] = '材料/化学/生物'
testA_labels_df[testA_labels_df['label_names'] == '马氏体/元件'] = '光电/电子/元件'
testA_labels_df[testA_labels_df['label_names'] == '基因/养殖'] = '农业/基因/养殖'
testA_labels_df['label'] = testA_labels_df['label_names'].apply(lambda x: label_list.index(x))
testA_labels_df['id'] = testA_df['id']

testA_labels_df[['id', 'label']].to_csv('../../results/prompt/4/submit (train6452_14epochs_promptclue).csv', encoding='utf8', sep=',', index=None)